---


<h1 style="text-align:center"> 
Lung Cancer Classification using CT Data (Smoke Detector)
</h1><style>

--- 
## Libraries


In [23]:
%pip install --upgrade matplotlib

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/7.8 MB 653.6 kB/s eta 0:00:12
   - -------------------------------------- 0.3/7.8 MB 2.1 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/7.8 MB 3.1 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/7.8 MB 3.4 MB/s eta 0:00:03
   ---- ----------------------------------- 0.9/7.8 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.0/7.8 MB 3.1 MB/s eta 0:00:03
   ----- ---------------------------------- 1.0/7.8 MB 2.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.2/7.8 MB 2.6 MB/s eta 0:00:03
   ------ --------------------------------- 1.2/7.8 MB 2.5 MB/s eta 0:00:03
   ------ --------------------------------- 1.3/7.8 MB 2.4 MB/s eta 0:00:03
   ------- --------------------

  You can safely remove it manually.
  You can safely remove it manually.


In [22]:
%pip install --help



Usage:   
  c:\Users\Daniela\Documents\Fisc\envs\base2\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\Daniela\Documents\Fisc\envs\base2\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\Daniela\Documents\Fisc\envs\base2\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\Daniela\Documents\Fisc\envs\base2\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\Daniela\Documents\Fisc\envs\base2\python.exe -m pip install [options] <archive url/path> ...

Description:
  Install packages from:
  
  - PyPI (and other indexes) using requirement specifiers.
  - VCS project urls.
  - Local project directories.
  - Local or remote source archives.
  
  pip also supports installing from "requirements files", which provide
  an easy way to specify a whole environment to be installed.

Install Options:
  -r, --requirement <file>    Install from the given re

In [1]:
import pandas as pd
import pylidc as pl
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np

In [2]:
print(pl.__file__)

c:\Users\Daniela\Desktop\Fac\3ºano\Lab_IACD\Project1\lung-cancer-classification\pylidc\__init__.py


---

## Importing XML files

In [10]:
plt.set_cmap('gray')

<Figure size 640x480 with 0 Axes>

In [6]:
#%pip install xlrd

In [11]:
nodule_counts_df = pd.read_excel('lidc-idri-nodule-counts-6-23-2015.xlsx')
nodule_counts_df
#maybe eliminar quarta e quinta coluna

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
pacient_data_df = pd.read_excel('tcia-diagnosis-data-2012-04-20.xls')
pacient_data_df 
#nem todos os pacientes têm historial registado (157 em 1010)

---


## Importing Annotations

In [2]:
annotation = pl.query(pl.Annotation).first()
print(annotation.scan.patient_id)

LIDC-IDRI-0078


In [6]:
pid = 'LIDC-IDRI-0001'
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()
print(len(scan.annotations))

4


In [7]:
nods = scan.cluster_annotations()

In [8]:
annotation.visualize_in_3d()

AttributeError: module 'matplotlib.cm' has no attribute 'cmap_d'

### Visualization of the annotated nodule

In [9]:
volume = annotation.scan.to_volume()
padding = [(30,10), (10,25), (0,0)]

mask = annotation.boolean_mask(pad=padding)
bbox = annotation.bbox(pad=padding)

fig,ax = plt.subplots(1,2,figsize=(5,3))

ax[0].imshow(vol[bbox][:,:,2], cmap=plt.cm.gray)
ax[0].axis('off')

ax[1].imshow(mask[:,:,2], cmap=plt.cm.gray)
ax[1].axis('off')

plt.tight_layout()
plt.savefig("../images/mask_bbox.png", bbox_inches="tight")
plt.show()

Loading dicom files ... This may take a moment.


RuntimeError: Couldn't find DICOM files for Scan(id=1,patient_id=LIDC-IDRI-0078) in C:/Users/Daniela/Desktop/Fac/3ºano/Lab_IACD/Project1/manifest-1727162347556/LIDC-IDRI\LIDC-IDRI-0078

### Surface Visualization

In [ ]:

# Verifica os colormaps disponíveis
print(plt.colormaps())

# Carregar a anotação de lobulação 5
ann = pl.query(pl.Annotation)\
        .filter(pl.Annotation.lobulation == 5).first()

# Usar a função visualize_in_3d com colormap válido
ann.visualize_in_3d(cmap='viridis') 

---

In [4]:
path_to_annotations = 'LIDC-XML-only'

In [6]:
path = 'C:/Users/Daniela/Desktop/Fac/3ºano/Lab_IACD/Project1/manifest-1727162347556/LIDC-IDRI'
f = open('C:/Users/Daniela/pylidc.conf', 'w')
f.write(f'[dicom]\npath = {path}\n\n')
f.close()

In [3]:
pid = 'LIDC-IDRI-0001'
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

In [ ]:
vol = scan.to_volume()
print(vol.shape) # (dim, dim, depth)

plt.figure(figsize=(5, 5))
plt.imshow(vol[:,:,0])
plt.show() # show the first-level slice

In [ ]:
#extract information from an XML file
def parse_xml(file):
    tree = ET.parse(file)
    root = tree.getroot()

    data = []
    for nodule in root